In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

#### Import model, training function 

In [3]:
from annsa.load_dataset import dataset_to_spectrakeys
from annsa.model_classes import build_dnn_model, compile_model
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Dataset Construction

#### Load dataset

In [37]:
dataset = np.load(os.path.join('dataset_generation',
                               'uenrichment_dataset_100000.npy'))
training_spectra = np.array(dataset.item()['spectra'], dtype='float64')
all_keys = np.array(dataset.item()['keys'])

## Model Construction

In [38]:
architecture_id = os.path.join('..',
                               'source-interdiction',
                               'final_training_notebooks',
                               'final-models',
                               'learningcurve-dnn-easy-final-features',
                              )

In [39]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.sigmoid
model_features.output_size = 1
model_features.metrics = ['mse']
model_features.learning_rate = model_features.learining_rate

In [40]:
dae_model = load_model(
    os.path.join('..',
                 'source-interdiction',
                 'final_training_notebooks',
                 'final-models-keras',
                 'daepretrain-easy-final.hdf5'))

## Train network

In [41]:
num_epochs = 200
patience = 10

# model_id_save_as = 'daednn-keras-easy'
model_id_save_as = 'daednn-easy'

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    patience=patience,
    mode='min',
    min_delta=0.01,
    restore_best_weights=True)

## Scale input data

In [42]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)

# Train

In [43]:
kf = KFold(n_splits=5)

k = 0
for train_index, validation_index in kf.split(training_spectra_scaled):
    model = compile_model(
        build_dnn_model,
        model_features)
    model_weights = model.get_weights()
    model_weights_updated = model_weights[:]
    model_weights_updated[0:2] = dae_model.get_weights()[0:2]
    model.set_weights(model_weights_updated)

    csv_logger = tf.keras.callbacks.CSVLogger(
        os.path.join('final-models-keras/',
        model_id_save_as+'_fold' + str(k) + '.log')

    output = model.fit(
        x=training_spectra_scaled[train_index],
        y=all_keys[train_index],
        epochs=num_epochs,
        verbose=1,
        validation_data=(training_spectra_scaled[validation_index],
                         all_keys[validation_index]),
        shuffle=True,
        callbacks=[earlystop_callback, csv_logger],
    )
    model.save(
        os.path.join('final-models-keras',
                     model_id_save_as + '_fold' + str(k) + '.hdf5')
    k += 1

Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 10s 126us/sample - loss: 1.0325 - mean_squared_error: 0.1541 - val_loss: 0.0613 - val_mean_squared_error: 0.0551
Epoch 2/200
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0689 - mean_squared_error: 0.0622 - val_loss: 0.0628 - val_mean_squared_error: 0.0574
Epoch 3/200
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0694 - mean_squared_error: 0.0627 - val_loss: 0.0650 - val_mean_squared_error: 0.0544
Epoch 4/200
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0687 - mean_squared_error: 0.0626 - val_loss: 0.0586 - val_mean_squared_error: 0.0543
Epoch 5/200
80000/80000 [==============================] - 9s 116us/sample - loss: 0.0647 - mean_squared_error: 0.0597 - val_loss: 0.0567 - val_mean_squared_error: 0.0499
Epoch 6/200
80000/80000 [==============================] - 9s 118us/sample - loss: 0.0609 - me

Epoch 17/200
80000/80000 [==============================] - 10s 120us/sample - loss: 0.0553 - mean_squared_error: 0.0507 - val_loss: 0.0493 - val_mean_squared_error: 0.0450
Epoch 18/200
80000/80000 [==============================] - 10s 120us/sample - loss: 0.0559 - mean_squared_error: 0.0515 - val_loss: 0.0538 - val_mean_squared_error: 0.0499
Epoch 19/200
80000/80000 [==============================] - 10s 120us/sample - loss: 0.0553 - mean_squared_error: 0.0509 - val_loss: 0.0468 - val_mean_squared_error: 0.0427
Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 11s 133us/sample - loss: 1.0743 - mean_squared_error: 0.2033 - val_loss: 0.0796 - val_mean_squared_error: 0.0686
Epoch 2/200
80000/80000 [==============================] - 10s 119us/sample - loss: 0.0670 - mean_squared_error: 0.0591 - val_loss: 0.0710 - val_mean_squared_error: 0.0683
Epoch 3/200
80000/80000 [==============================] - 9s 116us/sample - loss: 0.06